In [1]:
!pip install jsonlines

You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import os
import sys
import csv, jsonlines
import numpy as np
import copy
import random
import pandas as pd
import seaborn as sns

In [3]:
!python --version

Python 3.6.6 :: Anaconda, Inc.


### Data Exploration

In [4]:
data = pd.read_csv('../data/SampleData_3companies.csv')

In [5]:
data.head()

,Donation_Debit_Entry_ID,Company_ID,Cause_ID,User_Alias_Id,Donation_Country,Donation_Date,Donation_Amount,Donation_Currency,Cause_Name,Cause_Country,Cause_Category,Cause_Subcategory
0,132620506,1180,840-237245152,1670805,United States,2018-11-01 14:55:33.000000,40.0,USD,OCEAN CONSERVANCY INC,United States,"Environmental Quality, Protection, and Beautif...","Water Resource, Wetlands Conservation and Mana..."
1,132623765,1180,840-133039601,1760115,United States,2018-11-01 15:28:21.000000,50.0,USD,Alzheimer's Association National Office,United States,Voluntary Health Organizations and Medical Dis...,Alzheimer's Disease
2,132634483,1180,840-860832160,1680145,United States,2018-11-01 17:30:07.000000,100.0,USD,HALO HELPING ANIMALS LIVE ON,United States,Animal-Related,Animal Protection and Welfare
3,132634694,1180,840-237149453,1680145,United States,2018-11-01 17:32:03.000000,200.0,USD,ARIZONA ANIMAL WELFARE LEAGUE & SPCA,United States,Animal-Related,Animal Protection and Welfare
4,132635994,1180,840-421117292,1991295,United States,2018-11-01 17:45:01.000000,250.0,USD,PREVENT CHILD ABUSE IOWA,United States,Human Services,Alliance/Advocacy Organizations


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8137 entries, 0 to 8136
Data columns (total 12 columns):
Donation_Debit_Entry_ID    8137 non-null int64
Company_ID                 8137 non-null int64
Cause_ID                   8137 non-null object
User_Alias_Id              8137 non-null int64
Donation_Country           8137 non-null object
Donation_Date              8137 non-null object
Donation_Amount            8137 non-null float64
Donation_Currency          8137 non-null object
Cause_Name                 8137 non-null object
Cause_Country              8137 non-null object
Cause_Category             8137 non-null object
Cause_Subcategory          8137 non-null object
dtypes: float64(1), int64(3), object(8)
memory usage: 762.9+ KB


In [7]:
data.shape

(8137, 12)

### We need our input dataset in this format

* {'in0':[int(row[0])], 'in1':[int(row[1])], 'label':float(row[2])
* [{in0: 1,in1:2,label:4}, ...]
* in0 -> user_id -> User_Alias_Id
* in1 -> Item_Id -> Cause_ID
* Label -> How much the user likes the cause (Rating feature)
* Groupby user, take the average of the donation amount per user.
* Donation scaled per user
* the feature we are building per user will be calculated as follows:
* formula = individual_donation_amount/total_donation_amount_by_user


In [8]:
total_amount = data.groupby(by='User_Alias_Id')[['Donation_Amount']].sum()

In [9]:
total_amount.head()

,Donation_Amount
User_Alias_Id,
1649943,0.0
1649992,0.0
1650011,800.0
1650142,0.0
1650381,0.0


In [10]:
# total_amount = total_amount.reset_index()

In [11]:
total_amount.head()

,Donation_Amount
User_Alias_Id,
1649943,0.0
1649992,0.0
1650011,800.0
1650142,0.0
1650381,0.0


In [12]:
user_total_amount_map = total_amount.to_dict(orient='index')

In [13]:
user_total_amount_map

{1649943: {'Donation_Amount': 0.0},
 1649992: {'Donation_Amount': 0.0},
 1650011: {'Donation_Amount': 800.0},
 1650142: {'Donation_Amount': 0.0},
 1650381: {'Donation_Amount': 0.0},
 1650951: {'Donation_Amount': 10.0},
 1650958: {'Donation_Amount': 0.0},
 1650980: {'Donation_Amount': 0.0},
 1651048: {'Donation_Amount': 0.0},
 1651054: {'Donation_Amount': 100.0},
 1651061: {'Donation_Amount': 0.0},
 1656916: {'Donation_Amount': 1000.0},
 1661681: {'Donation_Amount': 0.0},
 1661780: {'Donation_Amount': 600.0},
 1661872: {'Donation_Amount': 250.0},
 1662102: {'Donation_Amount': 0.0},
 1662124: {'Donation_Amount': 400.0},
 1662169: {'Donation_Amount': 50.0},
 1662538: {'Donation_Amount': 0.0},
 1662545: {'Donation_Amount': 0.0},
 1662549: {'Donation_Amount': 50.0},
 1662570: {'Donation_Amount': 365.0},
 1662582: {'Donation_Amount': 0.0},
 1662640: {'Donation_Amount': 0.0},
 1662656: {'Donation_Amount': 0.0},
 1664061: {'Donation_Amount': 0.0},
 1664088: {'Donation_Amount': 0.0},
 1664108: 

In [14]:
data.head()

,Donation_Debit_Entry_ID,Company_ID,Cause_ID,User_Alias_Id,Donation_Country,Donation_Date,Donation_Amount,Donation_Currency,Cause_Name,Cause_Country,Cause_Category,Cause_Subcategory
0,132620506,1180,840-237245152,1670805,United States,2018-11-01 14:55:33.000000,40.0,USD,OCEAN CONSERVANCY INC,United States,"Environmental Quality, Protection, and Beautif...","Water Resource, Wetlands Conservation and Mana..."
1,132623765,1180,840-133039601,1760115,United States,2018-11-01 15:28:21.000000,50.0,USD,Alzheimer's Association National Office,United States,Voluntary Health Organizations and Medical Dis...,Alzheimer's Disease
2,132634483,1180,840-860832160,1680145,United States,2018-11-01 17:30:07.000000,100.0,USD,HALO HELPING ANIMALS LIVE ON,United States,Animal-Related,Animal Protection and Welfare
3,132634694,1180,840-237149453,1680145,United States,2018-11-01 17:32:03.000000,200.0,USD,ARIZONA ANIMAL WELFARE LEAGUE & SPCA,United States,Animal-Related,Animal Protection and Welfare
4,132635994,1180,840-421117292,1991295,United States,2018-11-01 17:45:01.000000,250.0,USD,PREVENT CHILD ABUSE IOWA,United States,Human Services,Alliance/Advocacy Organizations


In [15]:
data['total_donation_amount'] = data['User_Alias_Id'].apply(lambda x:user_total_amount_map.get(x)['Donation_Amount'])

In [16]:
data.head()

,Donation_Debit_Entry_ID,Company_ID,Cause_ID,User_Alias_Id,Donation_Country,Donation_Date,Donation_Amount,Donation_Currency,Cause_Name,Cause_Country,Cause_Category,Cause_Subcategory,total_donation_amount
0,132620506,1180,840-237245152,1670805,United States,2018-11-01 14:55:33.000000,40.0,USD,OCEAN CONSERVANCY INC,United States,"Environmental Quality, Protection, and Beautif...","Water Resource, Wetlands Conservation and Mana...",40.0
1,132623765,1180,840-133039601,1760115,United States,2018-11-01 15:28:21.000000,50.0,USD,Alzheimer's Association National Office,United States,Voluntary Health Organizations and Medical Dis...,Alzheimer's Disease,50.0
2,132634483,1180,840-860832160,1680145,United States,2018-11-01 17:30:07.000000,100.0,USD,HALO HELPING ANIMALS LIVE ON,United States,Animal-Related,Animal Protection and Welfare,300.0
3,132634694,1180,840-237149453,1680145,United States,2018-11-01 17:32:03.000000,200.0,USD,ARIZONA ANIMAL WELFARE LEAGUE & SPCA,United States,Animal-Related,Animal Protection and Welfare,300.0
4,132635994,1180,840-421117292,1991295,United States,2018-11-01 17:45:01.000000,250.0,USD,PREVENT CHILD ABUSE IOWA,United States,Human Services,Alliance/Advocacy Organizations,250.0


In [17]:
data.shape

(8137, 13)

#### Verification of apply lambda

In [18]:
user_total_amount_map.get(1656916)

{'Donation_Amount': 1000.0}

In [19]:
data.loc[data['User_Alias_Id']==1651048,:]

,Donation_Debit_Entry_ID,Company_ID,Cause_ID,User_Alias_Id,Donation_Country,Donation_Date,Donation_Amount,Donation_Currency,Cause_Name,Cause_Country,Cause_Category,Cause_Subcategory,total_donation_amount
1028,138494454,1180,840-340750345,1651048,United States,2018-11-26 19:47:08.000000,0.0,USD,HAVEN OF REST MINISTRIES INCORPORATED,United States,Religion-Related,Christian,0.0
1030,138494536,1180,840-135562351,1651048,United States,2018-11-26 19:49:23.000000,0.0,USD,The Salvation Army Eastern Territory,United States,Human Services,Salvation Army,0.0
1031,138494578,1180,840-133039601,1651048,United States,2018-11-26 19:50:47.000000,0.0,USD,Alzheimer's Association National Office,United States,Voluntary Health Organizations and Medical Dis...,Alzheimer's Disease,0.0
1032,138494620,1180,840-341369388,1651048,United States,2018-11-26 19:51:49.000000,0.0,USD,AKRON-CANTON REGIONAL FOODBANK,United States,"Food, Agriculture, and Nutrition","Food Banks, Food Pantries",0.0
1699,141222687,1180,840-135562351,1651048,United States,2018-12-12 10:46:41.000000,0.0,USD,The Salvation Army Eastern Territory,United States,Human Services,Salvation Army,0.0
1700,141222691,1180,840-341369388,1651048,United States,2018-12-12 10:48:01.000000,0.0,USD,AKRON-CANTON REGIONAL FOODBANK,United States,"Food, Agriculture, and Nutrition","Food Banks, Food Pantries",0.0
1701,141222699,1180,840-133039601,1651048,United States,2018-12-12 10:49:38.000000,0.0,USD,Alzheimer's Association National Office,United States,Voluntary Health Organizations and Medical Dis...,Alzheimer's Disease,0.0
2618,145621527,1180,840-341369388,1651048,United States,2018-12-26 14:40:00.000000,0.0,USD,AKRON-CANTON REGIONAL FOODBANK,United States,"Food, Agriculture, and Nutrition","Food Banks, Food Pantries",0.0
2694,145622752,1180,840-340750345,1651048,United States,2018-12-31 11:22:07.000000,0.0,USD,HAVEN OF REST MINISTRIES INCORPORATED,United States,Religion-Related,Christian,0.0


In [20]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8137 entries, 0 to 8136
Data columns (total 13 columns):
Donation_Debit_Entry_ID    8137 non-null int64
Company_ID                 8137 non-null int64
Cause_ID                   8137 non-null object
User_Alias_Id              8137 non-null int64
Donation_Country           8137 non-null object
Donation_Date              8137 non-null object
Donation_Amount            8137 non-null float64
Donation_Currency          8137 non-null object
Cause_Name                 8137 non-null object
Cause_Country              8137 non-null object
Cause_Category             8137 non-null object
Cause_Subcategory          8137 non-null object
total_donation_amount      8137 non-null float64
dtypes: float64(2), int64(3), object(8)
memory usage: 826.5+ KB


In [21]:
data.groupby(by='User_Alias_Id')[['Donation_Debit_Entry_ID']].count().head()

,Donation_Debit_Entry_ID
User_Alias_Id,
1649943,1
1649992,3
1650011,1
1650142,1
1650381,1


In [22]:
data.loc[data['Donation_Amount']==0.0,['User_Alias_Id']]['User_Alias_Id'].unique().shape

(1161,)

In [23]:
# sns.barplot(data=data,y='total_donation_amount',x='User_Alias_Id',ci=None);

In [24]:
data.head()

,Donation_Debit_Entry_ID,Company_ID,Cause_ID,User_Alias_Id,Donation_Country,Donation_Date,Donation_Amount,Donation_Currency,Cause_Name,Cause_Country,Cause_Category,Cause_Subcategory,total_donation_amount
0,132620506,1180,840-237245152,1670805,United States,2018-11-01 14:55:33.000000,40.0,USD,OCEAN CONSERVANCY INC,United States,"Environmental Quality, Protection, and Beautif...","Water Resource, Wetlands Conservation and Mana...",40.0
1,132623765,1180,840-133039601,1760115,United States,2018-11-01 15:28:21.000000,50.0,USD,Alzheimer's Association National Office,United States,Voluntary Health Organizations and Medical Dis...,Alzheimer's Disease,50.0
2,132634483,1180,840-860832160,1680145,United States,2018-11-01 17:30:07.000000,100.0,USD,HALO HELPING ANIMALS LIVE ON,United States,Animal-Related,Animal Protection and Welfare,300.0
3,132634694,1180,840-237149453,1680145,United States,2018-11-01 17:32:03.000000,200.0,USD,ARIZONA ANIMAL WELFARE LEAGUE & SPCA,United States,Animal-Related,Animal Protection and Welfare,300.0
4,132635994,1180,840-421117292,1991295,United States,2018-11-01 17:45:01.000000,250.0,USD,PREVENT CHILD ABUSE IOWA,United States,Human Services,Alliance/Advocacy Organizations,250.0


In [25]:
data.loc[data['Donation_Amount']==0.0,['User_Alias_Id']].index

Int64Index([  76,   77,   78,   79,   80,   81,   82,   83,   84,   85,
            ...
            8127, 8128, 8129, 8130, 8131, 8132, 8133, 8134, 8135, 8136],
           dtype='int64', length=1773)

In [26]:
data = data.drop(labels=data.loc[data['Donation_Amount']==0.0,['User_Alias_Id']].index)

In [27]:
data.shape

(6364, 13)

In [28]:
data.head()

,Donation_Debit_Entry_ID,Company_ID,Cause_ID,User_Alias_Id,Donation_Country,Donation_Date,Donation_Amount,Donation_Currency,Cause_Name,Cause_Country,Cause_Category,Cause_Subcategory,total_donation_amount
0,132620506,1180,840-237245152,1670805,United States,2018-11-01 14:55:33.000000,40.0,USD,OCEAN CONSERVANCY INC,United States,"Environmental Quality, Protection, and Beautif...","Water Resource, Wetlands Conservation and Mana...",40.0
1,132623765,1180,840-133039601,1760115,United States,2018-11-01 15:28:21.000000,50.0,USD,Alzheimer's Association National Office,United States,Voluntary Health Organizations and Medical Dis...,Alzheimer's Disease,50.0
2,132634483,1180,840-860832160,1680145,United States,2018-11-01 17:30:07.000000,100.0,USD,HALO HELPING ANIMALS LIVE ON,United States,Animal-Related,Animal Protection and Welfare,300.0
3,132634694,1180,840-237149453,1680145,United States,2018-11-01 17:32:03.000000,200.0,USD,ARIZONA ANIMAL WELFARE LEAGUE & SPCA,United States,Animal-Related,Animal Protection and Welfare,300.0
4,132635994,1180,840-421117292,1991295,United States,2018-11-01 17:45:01.000000,250.0,USD,PREVENT CHILD ABUSE IOWA,United States,Human Services,Alliance/Advocacy Organizations,250.0


In [29]:
data.loc[data['Donation_Amount']==0.0,['User_Alias_Id']]

,User_Alias_Id


#### Let's build the rating feature

In [30]:
data['rating'] = data['Donation_Amount']/data['total_donation_amount']

In [31]:
data.head()

,Donation_Debit_Entry_ID,Company_ID,Cause_ID,User_Alias_Id,Donation_Country,Donation_Date,Donation_Amount,Donation_Currency,Cause_Name,Cause_Country,Cause_Category,Cause_Subcategory,total_donation_amount,rating
0,132620506,1180,840-237245152,1670805,United States,2018-11-01 14:55:33.000000,40.0,USD,OCEAN CONSERVANCY INC,United States,"Environmental Quality, Protection, and Beautif...","Water Resource, Wetlands Conservation and Mana...",40.0,1.000000
1,132623765,1180,840-133039601,1760115,United States,2018-11-01 15:28:21.000000,50.0,USD,Alzheimer's Association National Office,United States,Voluntary Health Organizations and Medical Dis...,Alzheimer's Disease,50.0,1.000000
2,132634483,1180,840-860832160,1680145,United States,2018-11-01 17:30:07.000000,100.0,USD,HALO HELPING ANIMALS LIVE ON,United States,Animal-Related,Animal Protection and Welfare,300.0,0.333333
3,132634694,1180,840-237149453,1680145,United States,2018-11-01 17:32:03.000000,200.0,USD,ARIZONA ANIMAL WELFARE LEAGUE & SPCA,United States,Animal-Related,Animal Protection and Welfare,300.0,0.666667
4,132635994,1180,840-421117292,1991295,United States,2018-11-01 17:45:01.000000,250.0,USD,PREVENT CHILD ABUSE IOWA,United States,Human Services,Alliance/Advocacy Organizations,250.0,1.000000


In [32]:
data[['rating']].describe()

,rating
count,6364.000000
mean,0.273413
std,0.265523
min,0.010661
25%,0.111111
50%,0.119947
75%,0.500000
max,1.000000


#### Let's build a 'number_of_donations' feature
#### Let's sort the above dateframe by user_id so that all donations from the same user are in order
#### Let's build another columns consisting of a list of donation amounts per user - [40,50,100]

In [33]:
user_donation_count = data.groupby(by='User_Alias_Id')[['Donation_Debit_Entry_ID']].count()

In [34]:
user_count_map = user_donation_count.to_dict(orient='index')

In [35]:
user_count_map

{1650011: {'Donation_Debit_Entry_ID': 1},
 1650951: {'Donation_Debit_Entry_ID': 1},
 1651054: {'Donation_Debit_Entry_ID': 2},
 1656916: {'Donation_Debit_Entry_ID': 1},
 1661780: {'Donation_Debit_Entry_ID': 3},
 1661872: {'Donation_Debit_Entry_ID': 1},
 1662124: {'Donation_Debit_Entry_ID': 1},
 1662169: {'Donation_Debit_Entry_ID': 2},
 1662549: {'Donation_Debit_Entry_ID': 1},
 1662570: {'Donation_Debit_Entry_ID': 4},
 1664108: {'Donation_Debit_Entry_ID': 4},
 1664175: {'Donation_Debit_Entry_ID': 2},
 1664308: {'Donation_Debit_Entry_ID': 1},
 1664607: {'Donation_Debit_Entry_ID': 1},
 1664691: {'Donation_Debit_Entry_ID': 1},
 1664983: {'Donation_Debit_Entry_ID': 2},
 1664988: {'Donation_Debit_Entry_ID': 2},
 1665835: {'Donation_Debit_Entry_ID': 2},
 1666537: {'Donation_Debit_Entry_ID': 2},
 1667378: {'Donation_Debit_Entry_ID': 1},
 1667425: {'Donation_Debit_Entry_ID': 1},
 1667452: {'Donation_Debit_Entry_ID': 3},
 1667647: {'Donation_Debit_Entry_ID': 5},
 1667703: {'Donation_Debit_Entry_I

In [36]:
data['num_of_donations'] = data['User_Alias_Id'].apply(lambda x:user_count_map.get(x)['Donation_Debit_Entry_ID'])

In [37]:
data.head()

,Donation_Debit_Entry_ID,Company_ID,Cause_ID,User_Alias_Id,Donation_Country,Donation_Date,Donation_Amount,Donation_Currency,Cause_Name,Cause_Country,Cause_Category,Cause_Subcategory,total_donation_amount,rating,num_of_donations
0,132620506,1180,840-237245152,1670805,United States,2018-11-01 14:55:33.000000,40.0,USD,OCEAN CONSERVANCY INC,United States,"Environmental Quality, Protection, and Beautif...","Water Resource, Wetlands Conservation and Mana...",40.0,1.000000,1
1,132623765,1180,840-133039601,1760115,United States,2018-11-01 15:28:21.000000,50.0,USD,Alzheimer's Association National Office,United States,Voluntary Health Organizations and Medical Dis...,Alzheimer's Disease,50.0,1.000000,1
2,132634483,1180,840-860832160,1680145,United States,2018-11-01 17:30:07.000000,100.0,USD,HALO HELPING ANIMALS LIVE ON,United States,Animal-Related,Animal Protection and Welfare,300.0,0.333333,2
3,132634694,1180,840-237149453,1680145,United States,2018-11-01 17:32:03.000000,200.0,USD,ARIZONA ANIMAL WELFARE LEAGUE & SPCA,United States,Animal-Related,Animal Protection and Welfare,300.0,0.666667,2
4,132635994,1180,840-421117292,1991295,United States,2018-11-01 17:45:01.000000,250.0,USD,PREVENT CHILD ABUSE IOWA,United States,Human Services,Alliance/Advocacy Organizations,250.0,1.000000,1


In [38]:
data_sorted = data.sort_values(by='User_Alias_Id')

In [39]:
data_sorted.head(50)

,Donation_Debit_Entry_ID,Company_ID,Cause_ID,User_Alias_Id,Donation_Country,Donation_Date,Donation_Amount,Donation_Currency,Cause_Name,Cause_Country,Cause_Category,Cause_Subcategory,total_donation_amount,rating,num_of_donations
2604,145505496,1180,840-472214017,1650011,United States,2018-12-31 16:31:16.000000,800.00,USD,FURNISHING DIGNITY,United States,Housing and Shelter,Housing Support Services -- Other,800.0,1.000000,1
66,134442741,1180,840-742297116,1650951,United States,2018-11-08 16:27:30.000000,10.00,USD,AUSTIN POLICE OPERATION BLUE SANTA,United States,Human Services,Gift Distribution,10.0,1.000000,1
2498,144335678,1180,840-204631002,1651054,United States,2018-12-27 19:52:53.000000,50.00,USD,ONE OF A KIND PET RESCUE INC,United States,Animal-Related,Animal Protection and Welfare,100.0,0.500000,2
2352,143973900,1180,840-341091915,1651054,United States,2018-12-24 17:59:28.000000,50.00,USD,PORTAGE ANIMAL PROTECTIVE LEAGUE,United States,Animal-Related,Animal Protection and Welfare,100.0,0.500000,2
553,135647940,1180,840-770165029,1656916,United States,2018-11-16 18:19:23.000000,1000.00,USD,VENTURA COUNTY COMMUNITY FOUNDATION,United States,"Philanthropy, Voluntarism, and Grantmaking Fou...",Community Foundations,1000.0,1.000000,1
2424,144226605,1180,840-521802793,1661780,United States,2018-12-26 19:48:07.000000,250.00,USD,HO-HO-KUS EDUCATION FOUNDATION INC,United States,Education,Education N.E.C.,600.0,0.416667,3
577,135761874,1180,840-226094179,1661780,United States,2018-11-18 12:35:50.000000,250.00,USD,RAMAPO-BERGEN ANIMAL REFUGE INC,United States,Animal-Related,Animal Protection and Welfare,600.0,0.416667,3
1456,139447465,1180,840-060646623,1661780,United States,2018-12-07 20:06:37.000000,100.00,USD,FAIRFIELD UNIVERSITY,United States,Education,University or Technological Institute,600.0,0.166667,3
60,134438791,1180,840-222438932,1661872,United States,2018-11-08 14:20:25.000000,250.00,USD,RMHC OF WESTERN NEW YORK INC,United States,Health - General and Rehabilitative,"Patient Services - Entertainment, Recreation",250.0,1.000000,1
2749,145818608,1180,840-420680335,1662124,United States,2018-12-31 20:53:52.000000,400.00,USD,CORNELL COLLEGE,United States,Education,Undergraduate College (4-year),400.0,1.000000,1


In [40]:
#if rating > 0.2:
#	label = 1
#else:
#	label = 0

In [41]:
data.head()

,Donation_Debit_Entry_ID,Company_ID,Cause_ID,User_Alias_Id,Donation_Country,Donation_Date,Donation_Amount,Donation_Currency,Cause_Name,Cause_Country,Cause_Category,Cause_Subcategory,total_donation_amount,rating,num_of_donations
0,132620506,1180,840-237245152,1670805,United States,2018-11-01 14:55:33.000000,40.0,USD,OCEAN CONSERVANCY INC,United States,"Environmental Quality, Protection, and Beautif...","Water Resource, Wetlands Conservation and Mana...",40.0,1.000000,1
1,132623765,1180,840-133039601,1760115,United States,2018-11-01 15:28:21.000000,50.0,USD,Alzheimer's Association National Office,United States,Voluntary Health Organizations and Medical Dis...,Alzheimer's Disease,50.0,1.000000,1
2,132634483,1180,840-860832160,1680145,United States,2018-11-01 17:30:07.000000,100.0,USD,HALO HELPING ANIMALS LIVE ON,United States,Animal-Related,Animal Protection and Welfare,300.0,0.333333,2
3,132634694,1180,840-237149453,1680145,United States,2018-11-01 17:32:03.000000,200.0,USD,ARIZONA ANIMAL WELFARE LEAGUE & SPCA,United States,Animal-Related,Animal Protection and Welfare,300.0,0.666667,2
4,132635994,1180,840-421117292,1991295,United States,2018-11-01 17:45:01.000000,250.0,USD,PREVENT CHILD ABUSE IOWA,United States,Human Services,Alliance/Advocacy Organizations,250.0,1.000000,1


#### Add a repeated_donation feature by building a feature that incorporates the fact that the user has already donated to a particular cause before and add a weight to the rating based on this feature.

#### Tokenise the cause_id as object2vec only accepts integer-ids

In [42]:
unique_causes = data['Cause_ID'].unique().tolist()

In [43]:
cause_id_map = {}
for count,cause in enumerate(unique_causes):
    cause_id_map[cause] = count+1
    

In [247]:
len(cause_id_map.keys())

1199

In [248]:
cause_id_map

{'840-237245152': 1,
 '840-133039601': 2,
 '840-860832160': 3,
 '840-237149453': 4,
 '840-421117292': 5,
 '840-462917158': 6,
 '840-840421874': 7,
 '840-546001718': 8,
 '840-760620808': 9,
 '840-911800943': 10,
 '840-530196605': 11,
 '840-222406433': 12,
 '840-131760110': 13,
 '840-846038762': 14,
 '840-200049703': 15,
 '840-391773613': 16,
 '840-941416753': 17,
 '840-941156256': 18,
 '840-131623829': 19,
 '840-237353532': 20,
 '840-840869447': 21,
 '840-273486723': 22,
 '840-840405254': 23,
 '840-263781322': 24,
 '840-270911048': 25,
 '840-521218336': 26,
 '840-840862418': 27,
 '840-800532236': 28,
 '840-341926063': 29,
 '840-363203648': 30,
 '840-010579687': 31,
 '840-866051042': 32,
 '840-570422667': 33,
 '840-203369185': 34,
 '840-260403812': 35,
 '840-201780317': 36,
 '840-823840265': 37,
 '840-NCES8_01056755': 38,
 '840-841337868': 39,
 '840-742050021': 40,
 '840-840510404': 41,
 '840-942745572': 42,
 '840-941451515': 43,
 '840-742217350': 44,
 '840-711004773': 45,
 '840-74158748

In [45]:
data['cause_id_int'] = data['Cause_ID'].apply(lambda x:cause_id_map.get(x))

In [46]:
data.head()

,Donation_Debit_Entry_ID,Company_ID,Cause_ID,User_Alias_Id,Donation_Country,Donation_Date,Donation_Amount,Donation_Currency,Cause_Name,Cause_Country,Cause_Category,Cause_Subcategory,total_donation_amount,rating,num_of_donations,cause_id_int
0,132620506,1180,840-237245152,1670805,United States,2018-11-01 14:55:33.000000,40.0,USD,OCEAN CONSERVANCY INC,United States,"Environmental Quality, Protection, and Beautif...","Water Resource, Wetlands Conservation and Mana...",40.0,1.000000,1,1
1,132623765,1180,840-133039601,1760115,United States,2018-11-01 15:28:21.000000,50.0,USD,Alzheimer's Association National Office,United States,Voluntary Health Organizations and Medical Dis...,Alzheimer's Disease,50.0,1.000000,1,2
2,132634483,1180,840-860832160,1680145,United States,2018-11-01 17:30:07.000000,100.0,USD,HALO HELPING ANIMALS LIVE ON,United States,Animal-Related,Animal Protection and Welfare,300.0,0.333333,2,3
3,132634694,1180,840-237149453,1680145,United States,2018-11-01 17:32:03.000000,200.0,USD,ARIZONA ANIMAL WELFARE LEAGUE & SPCA,United States,Animal-Related,Animal Protection and Welfare,300.0,0.666667,2,4
4,132635994,1180,840-421117292,1991295,United States,2018-11-01 17:45:01.000000,250.0,USD,PREVENT CHILD ABUSE IOWA,United States,Human Services,Alliance/Advocacy Organizations,250.0,1.000000,1,5


In [47]:
data['cause_id_int'].unique().shape

(1199,)

#### Convert user_id to integer values less than 3000000 as that's the max integer_id allowed by object2vec
reference: https://docs.aws.amazon.com/sagemaker/latest/dg/object2vec-hyperparameters.html


input should be running sequnce of integers : 0 < int < enc0_vocab_size,
where enc0_vocab_size = 3000000 as a maximum
enc0_vocab_size is set to 3000 as it fails when set to a high number.

In [48]:
unique_users = data['User_Alias_Id'].unique().tolist()
user_map = {}
for count,user in enumerate(unique_users):
    user_map[user] = count+1

In [49]:
len(user_map.keys())

1740

In [50]:
data['user_id_int'] = data['User_Alias_Id'].apply(lambda x:user_map.get(x))

In [51]:
data['user_id_int'].unique().shape

(1740,)

#### Saving data

In [52]:
data_train = data.loc[:,['user_id_int','cause_id_int','rating']]

In [53]:
data_train.to_csv('../data/data_train.csv')

In [54]:
cause_data = data.loc[:,['cause_id_int','Cause_ID','Cause_Name','Cause_Country','Cause_Category','Cause_Subcategory']]

In [55]:
cause_data.to_csv('../data/cause_data.csv')

In [56]:
user_data = data.loc[:,['User_Alias_Id','user_id_int','Cause_ID','cause_id_int','Donation_Amount','Donation_Currency','total_donation_amount','num_of_donations']]

In [57]:
user_data.to_csv('../data/user_data.csv')

### Object2vec training

In [58]:
## some utility functions

def load_csv_data(filename, delimiter, verbose=True):
    """
    input: a file readable as csv and separated by a delimiter
    and has format users - movies - ratings - etc
    output: a list, where each row of the list is of the form
    {'in0':userID, 'in1':movieID, 'label':rating}
    """
    to_data_list = list()
    users = list()
    movies = list()
    ratings = list()
    unique_users = set()
    unique_movies = set()
    with open(filename, 'r') as csvfile:
        reader = csv.reader(csvfile, delimiter=delimiter)
        for count, row in enumerate(reader):
            #if count!=0:
            to_data_list.append({'in0':[int(row[0])], 'in1':[int(row[1])], 'label':float(row[2])})
            users.append(row[0])
            movies.append(row[1])
            ratings.append(float(row[2]))
            unique_users.add(row[0])
            unique_movies.add(row[1])
    if verbose:
        print("In file {}, there are {} ratings".format(filename, len(ratings)))
        print("The ratings have mean: {}, median: {}, and variance: {}".format(
                                            round(np.mean(ratings), 2), 
                                            round(np.median(ratings), 2), 
                                            round(np.var(ratings), 2)))
        print("There are {} unique users and {} unique movies".format(len(unique_users), len(unique_movies)))
    return to_data_list


def csv_to_augmented_data_dict(filename, delimiter):
    """
    Input: a file that must be readable as csv and separated by delimiter (to make columns)
    has format users - movies - ratings - etc
    Output:
      Users dictionary: keys as user ID's; each key corresponds to a list of movie ratings by that user
      Movies dictionary: keys as movie ID's; each key corresponds a list of ratings of that movie by different users
    """
    to_users_dict = dict() 
    to_movies_dict = dict()
    with open(filename, 'r') as csvfile:
        reader = csv.reader(csvfile, delimiter=delimiter)
        for count, row in enumerate(reader):
            #if count!=0:
            if row[0] not in to_users_dict:
                to_users_dict[row[0]] = [(row[1], row[2])]
            else:
                to_users_dict[row[0]].append((row[1], row[2]))
            if row[1] not in to_movies_dict:
                to_movies_dict[row[1]] = list(row[0])
            else:
                to_movies_dict[row[1]].append(row[0])
    return to_users_dict, to_movies_dict


def user_dict_to_data_list(user_dict):
    # turn user_dict format to data list format (acceptable to the algorithm)
    data_list = list()
    for user, movie_rating_list in user_dict.items():
        for movie, rating in movie_rating_list:
            data_list.append({'in0':[int(user)], 'in1':[int(movie)], 'label':float(rating)})
    return data_list

def divide_user_dicts(user_dict, sp_ratio_dict):
    """
    Input: A user dictionary, a ration dictionary
         - format of sp_ratio_dict = {'train':0.8, "test":0.2}
    Output: 
        A dictionary of dictionaries, with key corresponding to key provided by sp_ratio_dict
        and each key corresponds to a subdivded user dictionary
    """
    ratios = [val for _, val in sp_ratio_dict.items()]
    assert np.sum(ratios) == 1, "the sampling ratios must sum to 1!"
    divided_dict = {}
    for user, movie_rating_list in user_dict.items():
        sub_movies_ptr = 0
        sub_movies_list = []
        #movie_list, _ = zip(*movie_rating_list)
        #print(movie_list)
        for i, ratio in enumerate(ratios):
            if i < len(ratios)-1:
                sub_movies_ptr_end = sub_movies_ptr + int(len(movie_rating_list)*ratio)
                sub_movies_list.append(movie_rating_list[sub_movies_ptr:sub_movies_ptr_end])
                sub_movies_ptr = sub_movies_ptr_end
            else:
                sub_movies_list.append(movie_rating_list[sub_movies_ptr:])
        for subset_name in sp_ratio_dict.keys():
            if subset_name not in divided_dict:
                divided_dict[subset_name] = {user: sub_movies_list.pop(0)}
            else:
                #access sub-dictionary
                divided_dict[subset_name][user] = sub_movies_list.pop(0)
    
    return divided_dict

def write_csv_to_jsonl(jsonl_fname, csv_fname, csv_delimiter):
    """
    Input: a file readable as csv and separated by delimiter (to make columns)
        - has format users - movies - ratings - etc
    Output: a jsonline file converted from the csv file
    """
    with jsonlines.open(jsonl_fname, mode='w') as writer:
        with open(csv_fname, 'r') as csvfile:
            reader = csv.reader(csvfile, delimiter=csv_delimiter)
            for count, row in enumerate(reader):
                #print(row)
                #if count!=0:
                writer.write({'in0':[int(row[0])], 'in1':[int(row[1])], 'label':float(row[2])})
        print('Created {} jsonline file'.format(jsonl_fname))
                    
    
def write_data_list_to_jsonl(data_list, to_fname):
    """
    Input: a data list, where each row of the list is a Python dictionary taking form
    {'in0':userID, 'in1':movieID, 'label':rating}
    Output: save the list as a jsonline file
    """
    with jsonlines.open(to_fname, mode='w') as writer:
        for row in data_list:
            #print(row)
            writer.write({'in0':row['in0'], 'in1':row['in1'], 'label':row['label']})
    print("Created {} jsonline file".format(to_fname))

def data_list_to_inference_format(data_list, binarize=True, label_thres=3):
    """
    Input: a data list
    Output: test data and label, acceptable by SageMaker for inference
    """
    data_ = [({"in0":row['in0'], 'in1':row['in1']}, row['label']) for row in data_list]
    data, label = zip(*data_)
    infer_data = {"instances":data}
    if binarize:
        label = get_binarized_label(list(label), label_thres)
    return infer_data, label


def get_binarized_label(data_list, thres):
    """
    Input: data list
    Output: a binarized data list for recommendation task
    """
    for i, row in enumerate(data_list):
        if type(row) is dict:
            #if i < 10:
                #print(row['label'])
            if row['label'] > thres:
                #print(row)
                data_list[i]['label'] = 1
            else:
                data_list[i]['label'] = 0
        else:
            if row > thres:
                data_list[i] = 1
            else:
                data_list[i] = 0
    return data_list


In [59]:
from sklearn.model_selection import train_test_split

In [60]:
labels = data_train[['rating']]

In [61]:
data_train = data_train.drop(columns='rating')

In [62]:
data_train.head()

,user_id_int,cause_id_int
0,1,1
1,2,2
2,3,3
3,3,4
4,4,5


In [63]:
X_train,X_test,y_train,y_test = train_test_split(data_train,labels,test_size=0.3,random_state=42)

In [64]:
X_test.shape

(1910, 2)

In [65]:
#Combine X_train and y_train to create data in this format
#{'in0': [821], 'in1': [111], 'label': 4.0}   

In [66]:
labels.shape[0]

6364

In [67]:
def convert_to_o2v_dataformat(features,labels):
    """
    input:
    Takes in a dataframe of user_id and cause_id
    Takes in a dataframe of labels
    output: a list, where each row of the list is of the form
    {'in0':userID, 'in1':movieID, 'label':rating} 
    
    """
    l =[]
    for i in range(int(labels.shape[0])):
        d ={}
        d['in0']= [int(features.iloc[i,0])]
        d['in1']= [int(features.iloc[i,1])]
        d['label']=round(labels.iloc[i,0],2)
        l.append(d)
    return l 

In [68]:
train_data_list = convert_to_o2v_dataformat(X_train,y_train)

In [69]:
train_data_list

[{'in0': [855], 'in1': [636], 'label': 0.14},
 {'in0': [988], 'in1': [724], 'label': 0.11},
 {'in0': [1066], 'in1': [763], 'label': 0.18},
 {'in0': [807], 'in1': [671], 'label': 0.11},
 {'in0': [827], 'in1': [655], 'label': 0.11},
 {'in0': [816], 'in1': [675], 'label': 0.11},
 {'in0': [951], 'in1': [715], 'label': 0.04},
 {'in0': [1010], 'in1': [724], 'label': 0.11},
 {'in0': [1553], 'in1': [609], 'label': 0.1},
 {'in0': [1031], 'in1': [671], 'label': 0.11},
 {'in0': [1716], 'in1': [1155], 'label': 0.2},
 {'in0': [792], 'in1': [69], 'label': 0.16},
 {'in0': [246], 'in1': [92], 'label': 0.4},
 {'in0': [1000], 'in1': [724], 'label': 0.11},
 {'in0': [1013], 'in1': [724], 'label': 0.11},
 {'in0': [1374], 'in1': [636], 'label': 0.5},
 {'in0': [810], 'in1': [672], 'label': 0.11},
 {'in0': [1612], 'in1': [609], 'label': 0.5},
 {'in0': [256], 'in1': [135], 'label': 0.33},
 {'in0': [1165], 'in1': [636], 'label': 0.5},
 {'in0': [254], 'in1': [134], 'label': 1.0},
 {'in0': [538], 'in1': [404], 'l

In [70]:
test_data_list = convert_to_o2v_dataformat(X_test,y_test)

In [71]:
test_data_list

[{'in0': [1654], 'in1': [1039], 'label': 0.11},
 {'in0': [1119], 'in1': [647], 'label': 0.5},
 {'in0': [296], 'in1': [168], 'label': 0.33},
 {'in0': [739], 'in1': [637], 'label': 0.11},
 {'in0': [1058], 'in1': [741], 'label': 0.23},
 {'in0': [102], 'in1': [92], 'label': 0.33},
 {'in0': [21], 'in1': [29], 'label': 0.5},
 {'in0': [948], 'in1': [712], 'label': 0.11},
 {'in0': [601], 'in1': [422], 'label': 0.25},
 {'in0': [890], 'in1': [694], 'label': 0.11},
 {'in0': [1283], 'in1': [660], 'label': 0.25},
 {'in0': [429], 'in1': [380], 'label': 0.08},
 {'in0': [1458], 'in1': [647], 'label': 0.5},
 {'in0': [1490], 'in1': [927], 'label': 0.5},
 {'in0': [1302], 'in1': [636], 'label': 0.5},
 {'in0': [1678], 'in1': [1076], 'label': 0.04},
 {'in0': [1087], 'in1': [2], 'label': 0.1},
 {'in0': [1534], 'in1': [949], 'label': 0.08},
 {'in0': [1022], 'in1': [671], 'label': 0.11},
 {'in0': [182], 'in1': [92], 'label': 0.33},
 {'in0': [502], 'in1': [404], 'label': 1.0},
 {'in0': [1274], 'in1': [636], 'la

### Preparation of data for classification Task

In [72]:
#threshold passed is the median of ratings. 
#This is required to produce a balanced class
# data[['rating']].median(axis=0)
train_c = get_binarized_label(copy.deepcopy(train_data_list), 0.12)
test_c = get_binarized_label(copy.deepcopy(test_data_list), 0.12)

In [73]:
train_c

[{'in0': [855], 'in1': [636], 'label': 1},
 {'in0': [988], 'in1': [724], 'label': 0},
 {'in0': [1066], 'in1': [763], 'label': 1},
 {'in0': [807], 'in1': [671], 'label': 0},
 {'in0': [827], 'in1': [655], 'label': 0},
 {'in0': [816], 'in1': [675], 'label': 0},
 {'in0': [951], 'in1': [715], 'label': 0},
 {'in0': [1010], 'in1': [724], 'label': 0},
 {'in0': [1553], 'in1': [609], 'label': 0},
 {'in0': [1031], 'in1': [671], 'label': 0},
 {'in0': [1716], 'in1': [1155], 'label': 1},
 {'in0': [792], 'in1': [69], 'label': 1},
 {'in0': [246], 'in1': [92], 'label': 1},
 {'in0': [1000], 'in1': [724], 'label': 0},
 {'in0': [1013], 'in1': [724], 'label': 0},
 {'in0': [1374], 'in1': [636], 'label': 1},
 {'in0': [810], 'in1': [672], 'label': 0},
 {'in0': [1612], 'in1': [609], 'label': 1},
 {'in0': [256], 'in1': [135], 'label': 1},
 {'in0': [1165], 'in1': [636], 'label': 1},
 {'in0': [254], 'in1': [134], 'label': 1},
 {'in0': [538], 'in1': [404], 'label': 1},
 {'in0': [862], 'in1': [656], 'label': 0},
 {

In [74]:
test_c

[{'in0': [1654], 'in1': [1039], 'label': 0},
 {'in0': [1119], 'in1': [647], 'label': 1},
 {'in0': [296], 'in1': [168], 'label': 1},
 {'in0': [739], 'in1': [637], 'label': 0},
 {'in0': [1058], 'in1': [741], 'label': 1},
 {'in0': [102], 'in1': [92], 'label': 1},
 {'in0': [21], 'in1': [29], 'label': 1},
 {'in0': [948], 'in1': [712], 'label': 0},
 {'in0': [601], 'in1': [422], 'label': 1},
 {'in0': [890], 'in1': [694], 'label': 0},
 {'in0': [1283], 'in1': [660], 'label': 1},
 {'in0': [429], 'in1': [380], 'label': 0},
 {'in0': [1458], 'in1': [647], 'label': 1},
 {'in0': [1490], 'in1': [927], 'label': 1},
 {'in0': [1302], 'in1': [636], 'label': 1},
 {'in0': [1678], 'in1': [1076], 'label': 0},
 {'in0': [1087], 'in1': [2], 'label': 0},
 {'in0': [1534], 'in1': [949], 'label': 0},
 {'in0': [1022], 'in1': [671], 'label': 0},
 {'in0': [182], 'in1': [92], 'label': 1},
 {'in0': [502], 'in1': [404], 'label': 1},
 {'in0': [1274], 'in1': [636], 'label': 1},
 {'in0': [935], 'in1': [647], 'label': 0},
 {'

In [75]:
write_data_list_to_jsonl(train_c, '../data/train_c.jsonl')
write_data_list_to_jsonl(test_c, '../data/test_c.jsonl')

Created ../data/train_c.jsonl jsonline file
Created ../data/test_c.jsonl jsonline file


In [76]:
train_c_label = [row['label'] for row in train_c]
test_c_label = [row['label'] for row in test_c]

print("There are {} fraction of positive ratings in train_c.jsonl".format(
                                np.count_nonzero(train_c_label)/len(train_c_label)))
print("There are {} fraction of positive ratings in test_c.jsonl".format(
                                np.sum(test_c_label)/len(test_c_label)))

There are 0.479344409519533 fraction of positive ratings in train_c.jsonl
There are 0.4617801047120419 fraction of positive ratings in test_c.jsonl


In [77]:
import boto3 
import os
 
bucket = 'sagemaker-data-sou' # Customize your own bucket name
input_prefix = 'object2vec/cause-recommendation/input'
output_prefix = 'object2vec/cause-recommendation/output'

#### Upload data to S3 and make data paths

In [84]:
from sagemaker.session import s3_input

s3_client = boto3.client('s3')
input_paths = {}
output_path = os.path.join('s3://', bucket, output_prefix)

for data_name in ['train', 'test']:
    pre_key = os.path.join(input_prefix, 'rating', f'{data_name}')
    fname = '{}_c.jsonl'.format(data_name)
    data_path = os.path.join('s3://', bucket, pre_key, fname)
    s3_client.upload_file(os.path.join('../data',fname), bucket, os.path.join(pre_key, fname))
    input_paths[data_name] = s3_input(data_path, distribution='ShardedByS3Key', content_type='application/jsonlines')
    print('Uploaded {} data to {} and defined input path'.format(data_name, data_path))

print('Trained model will be saved at', output_path)

Uploaded train data to s3://sagemaker-data-sou/object2vec/cause-recommendation/input/rating/train/train_c.jsonl and defined input path
Uploaded test data to s3://sagemaker-data-sou/object2vec/cause-recommendation/input/rating/test/test_c.jsonl and defined input path
Trained model will be saved at s3://sagemaker-data-sou/object2vec/cause-recommendation/output


#### Get Object2Vec image

In [85]:
import sagemaker
from sagemaker import get_execution_role

sess = sagemaker.Session()

role = get_execution_role()
print(role)

## Get docker image of ObjectToVec algorithm
from sagemaker.amazon.amazon_estimator import get_image_uri
container = get_image_uri(boto3.Session().region_name, 'object2vec')
container

arn:aws:iam::017801987097:role/service-role/AmazonSageMaker-ExecutionRole-20200505T175453


'382416733822.dkr.ecr.us-east-1.amazonaws.com/object2vec:1'

#### training

In [90]:
from sagemaker.session import s3_input

hyperparameters_c = {
    "_kvstore": "device",
    "_num_gpus": "auto",
    "_num_kv_servers": "auto",
    "bucket_width": 0,
    "early_stopping_patience": 3, 
    "early_stopping_tolerance": 0.01,
    "enc0_cnn_filter_width": 3,
    "enc0_layers": "auto",
    "enc0_max_seq_len": 1,
    "enc0_network": "pooled_embedding",
    "enc0_token_embedding_dim": 300,
    "enc0_vocab_size": 3000,
    "enc1_cnn_filter_width": 3,
    "enc1_layers": "auto",
    "enc1_max_seq_len": 1,
    "enc1_network": "pooled_embedding",
    "enc1_token_embedding_dim": 300,
    "enc1_vocab_size": 3000,
    "enc_dim": 2048,
    "epochs": 20,
    "learning_rate": 0.001,
    "mini_batch_size": 2048,
    "mlp_activation": "relu",
    "mlp_dim": 1024,
    "mlp_layers": 1,
    "num_classes": 2,
    "optimizer": "adam",
    "output_layer": "softmax"
}

In [91]:
# get estimator
classifier = sagemaker.estimator.Estimator(container,
                                    role, 
                                    train_instance_count=1, 
                                    train_instance_type='ml.m5.xlarge',
                                    output_path=output_path,
                                    sagemaker_session=sess)

## set hyperparameters
classifier.set_hyperparameters(**hyperparameters_c)

## train, tune, and test the model
classifier.fit(input_paths)

2020-05-22 02:28:00 Starting - Starting the training job...
2020-05-22 02:28:02 Starting - Launching requested ML instances.........
2020-05-22 02:29:43 Starting - Preparing the instances for training...
2020-05-22 02:30:21 Downloading - Downloading input data
2020-05-22 02:30:21 Training - Downloading the training image.....
2020-05-22 02:31:14 Training - Training image download completed. Training in progress.Docker entrypoint called with argument(s): train
Running default environment configuration script
[05/22/2020 02:31:17 INFO 139712750188352] Reading default configuration from /opt/amazon/lib/python2.7/site-packages/algorithm/default-input.json: {u'comparator_list': u'hadamard, concat, abs_diff', u'enc0_vocab_file': u'', u'output_layer': u'softmax', u'enc0_cnn_filter_width': 3, u'epochs': 30, u'mlp_dim': 512, u'enc0_freeze_pretrained_embedding': u'true', u'mlp_layers': 2, u'_num_kv_servers': u'auto', u'weight_decay': 0, u'enc0_pretrained_embedding_file': u'', u'token_embedding_s

[05/22/2020 02:31:26 INFO 139712750188352] **************
[05/22/2020 02:31:26 INFO 139712750188352] Completed Epoch: 2, time taken: 0:00:02.633747
[05/22/2020 02:31:26 INFO 139712750188352] Epoch 2 Training metrics:   perplexity: 1.134 cross_entropy: 0.126 accuracy: 0.962 
[05/22/2020 02:31:26 INFO 139712750188352] #quality_metric: host=algo-1, epoch=2, train cross_entropy <loss>=0.126092431446
[05/22/2020 02:31:26 INFO 139712750188352] #quality_metric: host=algo-1, epoch=2, train accuracy <score>=0.96240234375
[05/22/2020 02:31:26 INFO 139712750188352] **************
#metrics {"Metrics": {"early_stop.time": {"count": 1, "max": 5.275964736938477, "sum": 5.275964736938477, "min": 5.275964736938477}, "update.time": {"count": 1, "max": 2639.8441791534424, "sum": 2639.8441791534424, "min": 2639.8441791534424}}, "EndTime": 1590114686.726757, "Dimensions": {"Host": "algo-1", "Operation": "training", "Algorithm": "ObjectToVec"}, "StartTime": 1590114683.870635}

[05/22/2020 02:31:26 INFO 1397

[05/22/2020 02:31:38 INFO 139712750188352] **************
[05/22/2020 02:31:38 INFO 139712750188352] Completed Epoch: 6, time taken: 0:00:02.630201
[05/22/2020 02:31:38 INFO 139712750188352] Epoch 6 Training metrics:   perplexity: 1.008 cross_entropy: 0.008 accuracy: 0.998 
[05/22/2020 02:31:38 INFO 139712750188352] #quality_metric: host=algo-1, epoch=6, train cross_entropy <loss>=0.00782237372672
[05/22/2020 02:31:38 INFO 139712750188352] #quality_metric: host=algo-1, epoch=6, train accuracy <score>=0.99755859375
[05/22/2020 02:31:38 INFO 139712750188352] **************
[05/22/2020 02:31:38 INFO 139712750188352] patience losses: [0.04946482057372729, 0.023708253478010494, 0.01385283408065637]
[05/22/2020 02:31:38 INFO 139712750188352] min patience losses: 0.0138528340807
[05/22/2020 02:31:38 INFO 139712750188352] current loss: 0.00782237372672
[05/22/2020 02:31:38 INFO 139712750188352] absolute loss difference: 0.00603046035394
[05/22/2020 02:31:38 INFO 139712750188352] Bad epoch: los


2020-05-22 02:32:02 Uploading - Uploading generated training model
2020-05-22 02:32:02 Completed - Training job completed
Training seconds: 107
Billable seconds: 107


In [93]:
from sagemaker.predictor import json_serializer, json_deserializer

classification_model = classifier.create_model(
                        serializer=json_serializer,
                        deserializer=json_deserializer,
                        content_type='application/json')

predictor = classification_model.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

-----------------!

In [98]:
#label_thres set to 0.12 which is the median of the ratings
test_c_data, test_c_label = data_list_to_inference_format(copy.deepcopy(test_data_list), 
                                                            label_thres=0.12, binarize=True)
predictions = predictor.predict(test_c_data)

In [99]:
test_c_data

{'instances': ({'in0': [1654], 'in1': [1039]},
  {'in0': [1119], 'in1': [647]},
  {'in0': [296], 'in1': [168]},
  {'in0': [739], 'in1': [637]},
  {'in0': [1058], 'in1': [741]},
  {'in0': [102], 'in1': [92]},
  {'in0': [21], 'in1': [29]},
  {'in0': [948], 'in1': [712]},
  {'in0': [601], 'in1': [422]},
  {'in0': [890], 'in1': [694]},
  {'in0': [1283], 'in1': [660]},
  {'in0': [429], 'in1': [380]},
  {'in0': [1458], 'in1': [647]},
  {'in0': [1490], 'in1': [927]},
  {'in0': [1302], 'in1': [636]},
  {'in0': [1678], 'in1': [1076]},
  {'in0': [1087], 'in1': [2]},
  {'in0': [1534], 'in1': [949]},
  {'in0': [1022], 'in1': [671]},
  {'in0': [182], 'in1': [92]},
  {'in0': [502], 'in1': [404]},
  {'in0': [1274], 'in1': [636]},
  {'in0': [935], 'in1': [647]},
  {'in0': [783], 'in1': [656]},
  {'in0': [951], 'in1': [714]},
  {'in0': [250], 'in1': [220]},
  {'in0': [964], 'in1': [723]},
  {'in0': [931], 'in1': [609]},
  {'in0': [1422], 'in1': [899]},
  {'in0': [1039], 'in1': [727]},
  {'in0': [874], 

In [100]:
test_c_label

[0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,


In [101]:
predictions

{'predictions': [{'scores': [0.012466032058001, 0.987533926963806]},
  {'scores': [2.082980018e-06, 0.999997973442078]},
  {'scores': [1.3879214748e-05, 0.999986171722412]},
  {'scores': [0.99999988079071, 1.17394293e-07]},
  {'scores': [0.989644527435303, 0.010355438105762]},
  {'scores': [4.5370946e-08, 1.0]},
  {'scores': [0.008887503296137, 0.991112530231476]},
  {'scores': [1.0, 3.5965794e-08]},
  {'scores': [8.12190535e-07, 0.999999165534973]},
  {'scores': [1.0, 4.2011397e-08]},
  {'scores': [1.13309021e-07, 0.99999988079071]},
  {'scores': [1.528855933e-06, 0.999998450279236]},
  {'scores': [3.82142645e-07, 0.999999642372131]},
  {'scores': [2.9053146e-07, 0.999999761581421]},
  {'scores': [7.74121645e-07, 0.999999284744263]},
  {'scores': [0.99999988079071, 1.05322727e-07]},
  {'scores': [0.999504685401916, 0.00049532414414]},
  {'scores': [0.999998092651367, 1.866913408e-06]},
  {'scores': [0.999996662139893, 3.368897751e-06]},
  {'scores': [4.13796471e-07, 0.999999642372131]

In [103]:
def get_class_accuracy(res, labels, thres):
    """
    Converts scores to classification output
    """
    if type(res) is dict:
        res = res['predictions']
    assert len(res)==len(labels), 'result and label length mismatch!'
    accuracy = 0
    for row, label in zip(res, labels):
        if type(row) is dict:
            if row['scores'][1] > thres:
                prediction = 1
            else: 
                prediction = 0
            if label > thres:
                label = 1
            else:
                label = 0
            accuracy += 1 - (prediction - label)**2
    return accuracy / float(len(res))

print("The accuracy on the binarized validation set is %.3f" %get_class_accuracy(predictions, test_c_label, 0.12))

The accuracy on the binarized validation set is 0.953


#### Cause retreival in the embedding space

In [155]:
def get_movie_embedding_dict(movie_ids, trained_model):
    """
    https://docs.aws.amazon.com/sagemaker/latest/dg/object2vec-encoder-embeddings.html
    """
    input_instances = list()
    for s_id in movie_ids:
        input_instances.append({'in1': [s_id]})
    data = {'instances': input_instances}
    movie_embeddings = trained_model.predict(data)
    print("Number of embeddings")
    print(len(movie_embeddings['predictions']))
    embedding_dict = {}
    for s_id, row in zip(movie_ids, movie_embeddings['predictions']):
        embedding_dict[s_id] = np.array(row['embeddings'])
    return embedding_dict


            
def get_nn_of_movie(movie_id, candidate_movie_ids, embedding_dict):
    movie_emb = embedding_dict[movie_id]
    min_dist = float('Inf')
    best_id = list()
    for idx, m_id in enumerate(candidate_movie_ids):
        candidate_emb = embedding_dict[m_id]
        curr_dist = np.linalg.norm(candidate_emb - movie_emb)
        if curr_dist < min_dist:
            if len(best_id) >10:
                break
            best_id.append(m_id)
            min_dist = curr_dist
    return best_id, min_dist

In [156]:
unique_causes_list = data['cause_id_int'].unique().tolist()

In [157]:
# unique_causes_list

In [158]:
# train_data_list = load_csv_data(train_path, '\t', verbose=False)
# unique_movie_ids = unique_causes
embedding_dict = get_movie_embedding_dict(unique_causes_list, predictor)
candidate_causes = unique_causes_list.copy()
# unique_causes

Number of embeddings
1199


In [159]:
embedding_dict

{1: array([-0.00693873,  0.01543833,  0.02512959, ...,  0.00082226,
        -0.01685345, -0.02462569]),
 2: array([-0.00796028,  0.0278649 , -0.01694334, ...,  0.01740917,
        -0.02306035, -0.032355  ]),
 3: array([-0.00368585, -0.00880166,  0.01720645, ..., -0.02132149,
         0.01037189, -0.02221254]),
 4: array([ 0.01753447,  0.01454836,  0.01710996, ...,  0.01992301,
         0.00921615, -0.0205167 ]),
 5: array([ 0.00215365, -0.03038621, -0.01448082, ..., -0.02247252,
         0.00592352, -0.01969361]),
 6: array([-0.03094859,  0.02632939,  0.00776981, ...,  0.01710917,
        -0.00524371,  0.01484586]),
 7: array([-0.01706574, -0.02582515,  0.03073588, ..., -0.0015836 ,
         0.0114716 ,  0.01771497]),
 8: array([ 0.01379453,  0.00873102, -0.00549097, ...,  0.02013628,
        -0.00471418,  0.00553246]),
 9: array([ 0.00048126, -0.02173861,  0.02811903, ..., -0.0190077 ,
         0.01120467, -0.03024639]),
 10: array([-0.00822467, -0.03374162,  0.02148913, ..., -0.01829

In [160]:
# pd.DataFrame.from_dict(embedding_dict).describe()

,1,2,3,4,5,6,7,8,9,10,...,1190,1191,1192,1193,1194,1195,1196,1197,1198,1199
count,2048.000000,2048.000000,2048.000000,2048.000000,2048.000000,2048.000000,2048.000000,2048.000000,2048.000000,2048.000000,...,2048.000000,2048.000000,2048.000000,2048.000000,2048.000000,2048.000000,2048.000000,2048.000000,2048.000000,2048.000000
mean,-0.000499,0.000857,-0.000192,0.000204,0.000509,0.000253,-0.000651,-0.000051,0.000424,0.000032,...,-0.000363,0.000253,-0.000217,-0.000335,0.000121,0.000774,0.000766,0.000066,-0.000092,-0.000199
std,0.022285,0.020785,0.020776,0.022108,0.021376,0.022595,0.021317,0.019981,0.021014,0.020981,...,0.022149,0.019721,0.021826,0.020955,0.021060,0.019822,0.023618,0.019610,0.019884,0.021934
min,-0.047291,-0.047784,-0.041819,-0.043916,-0.045033,-0.045760,-0.043511,-0.034471,-0.043476,-0.042062,...,-0.047450,-0.034458,-0.042853,-0.043139,-0.042672,-0.034446,-0.048083,-0.034429,-0.034404,-0.044237
25%,-0.017598,-0.016057,-0.017435,-0.017382,-0.016999,-0.017466,-0.018319,-0.017386,-0.016678,-0.017239,...,-0.017514,-0.016550,-0.017867,-0.017642,-0.016903,-0.016292,-0.016359,-0.016847,-0.017191,-0.018311
50%,-0.001100,0.001334,0.000393,0.000320,0.001634,0.001025,-0.001670,0.000016,0.000452,0.000324,...,-0.000624,0.000091,-0.000540,-0.000158,0.000254,0.001024,0.001130,0.000167,0.000207,-0.000922
75%,0.016996,0.018385,0.017246,0.018359,0.017384,0.017911,0.017440,0.017590,0.018410,0.016742,...,0.016864,0.016804,0.017845,0.017053,0.016822,0.017858,0.017960,0.016840,0.017146,0.017883
max,0.046937,0.043537,0.042198,0.043786,0.044894,0.046196,0.043158,0.034428,0.042495,0.042185,...,0.046367,0.034464,0.042795,0.043264,0.043434,0.034424,0.048320,0.034471,0.034402,0.044011


In [170]:
movie_id_to_examine = 56

In [171]:
candidate_causes.remove(movie_id_to_examine)
best_id, min_dist = get_nn_of_movie(movie_id_to_examine, candidate_causes, embedding_dict)

In [172]:
best_id

[1, 4, 17, 20, 209]

In [145]:
min_dist

1.2306640760909169

In [245]:
def get_cause_details(cause_id_list=None):
    """
    Returns a dataframe of cause details
    """
    df = pd.read_csv('../data/cause_data.csv')
    new_df = pd.DataFrame(columns=df.columns)
    for i in cause_id_list:
        new_df = new_df.append(df.loc[df['cause_id_int']==i,:])
    return new_df

In [246]:
get_cause_details([1, 4, 17, 20, 209])

,Unnamed: 0,cause_id_int,Cause_ID,Cause_Name,Cause_Country,Cause_Category,Cause_Subcategory
0,0,1,840-237245152,OCEAN CONSERVANCY INC,United States,"Environmental Quality, Protection, and Beautif...","Water Resource, Wetlands Conservation and Mana..."
1026,2155,1,840-237245152,OCEAN CONSERVANCY INC,United States,"Environmental Quality, Protection, and Beautif...","Water Resource, Wetlands Conservation and Mana..."
3,3,4,840-237149453,ARIZONA ANIMAL WELFARE LEAGUE & SPCA,United States,Animal-Related,Animal Protection and Welfare
18,18,17,840-941416753,ST MATTHEWS EPISCOPAL DAY SCHOOL,United States,Education,"Elementary, Secondary Education, K - 12"
21,21,20,840-237353532,ST MARYS FOOD BANK ALLIANCE,United States,"Food, Agriculture, and Nutrition","Food Banks, Food Pantries"
77,241,20,840-237353532,ST MARYS FOOD BANK ALLIANCE,United States,"Food, Agriculture, and Nutrition","Food Banks, Food Pantries"
87,251,20,840-237353532,ST MARYS FOOD BANK ALLIANCE,United States,"Food, Agriculture, and Nutrition","Food Banks, Food Pantries"
296,580,20,840-237353532,ST MARYS FOOD BANK ALLIANCE,United States,"Food, Agriculture, and Nutrition","Food Banks, Food Pantries"
455,903,20,840-237353532,ST MARYS FOOD BANK ALLIANCE,United States,"Food, Agriculture, and Nutrition","Food Banks, Food Pantries"
516,969,20,840-237353532,ST MARYS FOOD BANK ALLIANCE,United States,"Food, Agriculture, and Nutrition","Food Banks, Food Pantries"


In [238]:
df_cause = pd.read_csv('../data/cause_data.csv',index_col='cause_id_int')
# var2 = [1, 4, 17, 20, 209]
# new_df = pd.DataFrame(columns=df_cause.columns)
# df_cause.iloc[1]
# # for i in var2:
# #     new_df = new_df.append(df_cause.iloc[i],ignore_index=True)
df_cause

,Unnamed: 0,Cause_ID,Cause_Name,Cause_Country,Cause_Category,Cause_Subcategory
cause_id_int,,,,,,
1,0,840-237245152,OCEAN CONSERVANCY INC,United States,"Environmental Quality, Protection, and Beautif...","Water Resource, Wetlands Conservation and Mana..."
2,1,840-133039601,Alzheimer's Association National Office,United States,Voluntary Health Organizations and Medical Dis...,Alzheimer's Disease
3,2,840-860832160,HALO HELPING ANIMALS LIVE ON,United States,Animal-Related,Animal Protection and Welfare
4,3,840-237149453,ARIZONA ANIMAL WELFARE LEAGUE & SPCA,United States,Animal-Related,Animal Protection and Welfare
5,4,840-421117292,PREVENT CHILD ABUSE IOWA,United States,Human Services,Alliance/Advocacy Organizations
6,5,840-462917158,*SUN FOUNDATION,United States,Mental Health and Crisis Intervention,Fund Raising and/or Fund Distribution
6,6,840-462917158,*SUN FOUNDATION,United States,Mental Health and Crisis Intervention,Fund Raising and/or Fund Distribution
7,7,840-840421874,COLORADO ACADEMY,United States,Education,"Primary, Elementary Schools"
8,8,840-546001718,*COLLEGE OF WILLIAM AND MARY,United States,Education,Higher Education Institutions


In [234]:
new_df

,Unnamed: 0,cause_id_int,Cause_ID,Cause_Name,Cause_Country,Cause_Category,Cause_Subcategory
